<a href="https://colab.research.google.com/github/ipinmi/grounded-vqa-fm/blob/detection/clip_vqa_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip vcr1annots.zip

Archive:  vcr1annots.zip
   creating: vcr1annots/
  inflating: vcr1annots/train.jsonl  
  inflating: vcr1annots/cocoontology.json  
  inflating: vcr1annots/val.jsonl    


In [ ]:
!unzip vcr1images.zip

In [7]:
!git clone https://ipinmi:@github.com/ipinmi/grounded-vqa-fm.git

Cloning into 'grounded-vqa-fm'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 29 (delta 5), reused 17 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 52.24 KiB | 1.93 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [8]:
!cd grounded-vqa-fm && git checkout detection

Branch 'detection' set up to track remote branch 'detection' from 'origin'.
Switched to a new branch 'detection'
